In [1]:
#Crea las tablas en MySQL

import MySQLdb  

connection = MySQLdb.connect(
    host="mysql",          
    user="my_app_user",
    passwd="my_app_pass",  
    db="my_app_db",
    port=3306
)

cursor = connection.cursor()

# Ejecutar tus queries
cursor.execute("DROP TABLE IF EXISTS penguins_raw;")
cursor.execute("""
CREATE TABLE penguins_raw (
    species VARCHAR(50) NULL,
    island VARCHAR(50) NULL,
    bill_length_mm DOUBLE NULL,
    bill_depth_mm DOUBLE NULL,
    flipper_length_mm DOUBLE NULL,
    body_mass_g DOUBLE NULL,
    sex VARCHAR(10) NULL,
    year INT NULL
);
""")

cursor.execute("DROP TABLE IF EXISTS penguins_clean;")
cursor.execute("""
CREATE TABLE penguins_clean (
    species INT NULL,
    bill_length_mm DOUBLE NULL,
    bill_depth_mm DOUBLE NULL,
    flipper_length_mm DOUBLE NULL,
    body_mass_g DOUBLE NULL,
    year INT NULL,
    island_Biscoe INT NULL,
    island_Dream INT NULL,
    island_Torgersen INT NULL,
    sex_female INT NULL,
    sex_male INT NULL
);
""")

connection.commit()
connection.close()

hola


In [2]:
#Inserta la información cruda 

import palmerpenguins as pp
from palmerpenguins import load_penguins
import pandas as pd
"""
Función para insertar datos en la tabla MySQL
"""
TABLE_NAME = "penguins_raw"
df = pp.load_penguins()


connection = MySQLdb.connect(
    host="mysql",          
    user="my_app_user",
    passwd="my_app_pass",  
    db="my_app_db",
    port=3306
)

cursor = connection.cursor()

for _, row in df.iterrows():
    sql = f"""
    INSERT INTO {TABLE_NAME} 
    (species, island, bill_length_mm, bill_depth_mm, flipper_length_mm, body_mass_g, sex, year)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    values = (
        row["species"],
        row["island"],
        None if pd.isna(row["bill_length_mm"]) else float(row["bill_length_mm"]),
        None if pd.isna(row["bill_depth_mm"]) else float(row["bill_depth_mm"]),
        None if pd.isna(row["flipper_length_mm"]) else float(row["flipper_length_mm"]),
        None if pd.isna(row["body_mass_g"]) else float(row["body_mass_g"]),
        row["sex"] if pd.notna(row["sex"]) else None,
        int(row["year"]),
    )
    
    cursor.execute(sql, values)

connection.commit()
connection.close()

In [3]:
#Trae la información desde MySQL
conn = MySQLdb.connect(
    host="mysql",         
    user="my_app_user",
    passwd="my_app_pass",  
    db="my_app_db",
    port=3306
)

query = "SELECT * FROM penguins_raw"
df = pd.read_sql(query, conn)

/tmp/ipykernel_45/2044866645.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [4]:
#Limpia los datos 

from sklearn.preprocessing import OneHotEncoder
import numpy as np

df[df.isna().any(axis=1)]
df.dropna(inplace=True)
categorical_cols = ['sex','island']
encoder = OneHotEncoder(handle_unknown='ignore')
x = df.drop(columns=['species'])
y = df['species']
x_encoded = encoder.fit_transform(x[categorical_cols])
X_numeric = x.drop(columns=categorical_cols)
X_final = np.hstack((X_numeric.values, x_encoded.toarray()))

df_encoded = pd.get_dummies(df, columns=['island','sex'])
bool_cols = df_encoded.select_dtypes(include='bool').columns
df_encoded[bool_cols] = df_encoded[bool_cols].astype(int)
df_encoded.head()
df_encoded['species'] = df_encoded['species'].apply(lambda x: 
                        1 if x == 'Adelie' else 
                        2 if x == 'Chinstrap' else 
                        3 if x == 'Gentoo' else 
                        None)

In [5]:
# Inserta la información limpia en MySQL
TABLE_NAME = "penguins_clean"

connection = MySQLdb.connect(
    host="mysql",          
    user="my_app_user",
    passwd="my_app_pass",  
    db="my_app_db",
    port=3306
)

cursor = connection.cursor()

insert_sql = """
    INSERT INTO penguins_clean (
        species, bill_length_mm, bill_depth_mm, flipper_length_mm, 
        body_mass_g, year, island_Biscoe, island_Dream, island_Torgersen, 
        sex_female, sex_male
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Genera los valores a insertar
values = [
    (
        int(row["species"]),
        None if pd.isna(row["bill_length_mm"]) else float(row["bill_length_mm"]),
        None if pd.isna(row["bill_depth_mm"]) else float(row["bill_depth_mm"]),
        None if pd.isna(row["flipper_length_mm"]) else float(row["flipper_length_mm"]),
        None if pd.isna(row["body_mass_g"]) else float(row["body_mass_g"]),
        int(row["year"]),
        int(row["island_Biscoe"]),
        int(row["island_Dream"]),
        int(row["island_Torgersen"]),
        int(row["sex_female"]),
        int(row["sex_male"]),
    )
    for _, row in df_encoded.iterrows()
]

# Inserta múltiples filas
cursor.executemany(insert_sql, values)

# Confirma los cambios
connection.commit()

# Cierra conexión
cursor.close()
connection.close()


In [6]:
#Trae la data limpia para entrenar el modelo
conn = MySQLdb.connect(
    host="mysql",          
    user="my_app_user",
    passwd="my_app_pass",  
    db="my_app_db",
    port=3306
)

query = "SELECT * FROM penguins_clean"
df_limpio = pd.read_sql(query, conn)

/tmp/ipykernel_45/199009257.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_limpio = pd.read_sql(query, conn)


In [7]:
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# ==============================
# 1. Conectar con tu MLflow server
# ==============================
mlflow.set_tracking_uri("http://10.43.100.98:5005")  # Ajusta si tu server corre en otro host/puerto

# ==============================
# 2. Preparar dataset
# ==============================
df = df_limpio  # <-- tu DataFrame limpio
X = df.drop("species", axis=1)
y = df["species"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ==============================
# 3. Entrenar modelo
# ==============================
model = LogisticRegression(max_iter=5000)
model.fit(X_train, y_train)

# ==============================
# 4. Definir experimento
# ==============================
mlflow.set_experiment("lucio_experiment3")

# ==============================
# 5. Loggear y registrar el modelo
# ==============================
with mlflow.start_run(run_name="logistic_regression_run") as run:
    # Log del modelo como artifact
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model"
    )

    # URI del modelo guardado en este run
    run_id = run.info.run_id
    model_uri = f"runs:/{run_id}/model"

    # Registrar en el Model Registry del server
    mlflow.register_model(
        model_uri=model_uri,
        name="lucio_model3"
    )

print("✅ Modelo entrenado, logueado y registrado en MLflow")

Registered model 'lucio_model3' already exists. Creating a new version of this model...
2025/09/12 04:09:37 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: lucio_model3, version 2
Created version '2' of model 'lucio_model3'.


✅ Modelo entrenado, logueado y registrado en MLflow


In [8]:
from mlflow.tracking import MlflowClient

# Conexión al server
mlflow.set_tracking_uri("http://10.43.100.98:5005")
client = MlflowClient()

# Actualizar a producción
model_name = "lucio_model3"
model_version = 1  # la primera versión que acabas de crear

client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage="Production",
    archive_existing_versions=True  # mueve versiones anteriores a Archived
)

print(f"✅ Modelo {model_name} v{model_version} promovido a Production")

✅ Modelo lucio_model3 v1 promovido a Production
